# lesson01选做题

### 是否匹配

In [4]:
import random
choice = random.choice
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_match(pattern = 'My ?X told me something',saying= 'My mother is fuking you'):
    if not pattern or not saying: return []
    if is_variable(pattern[0]):
        try:
            return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
        except TypeError:
            return False
    else:
        if pattern[0] != saying[0]:return False
        else:
            return pat_match(pattern[1:],saying[1:])

pat_match('?X want holiday'.split(),"who want holiday".split())

def pat_to_dict(patterns):
    return {k:v for k,v in patterns}

def subsitite(rule,parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0],rule[0])] + subsitite(rule[1:],parsed_rules)

In [5]:
got_patterns = pat_match("I wangt ?X".split(),"I wangt iPhone".split())
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

### 定义规则，进行匹配

In [6]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

def get_response(saying,defined_patterns):
    for rule in defined_patterns.keys():
        if pat_match(rule.split(),saying.split()):
            ans = choice(defined_patterns[rule])
            got_patterns = pat_match(rule.split(),saying.split())
            continue
    try:
        return ' '.join(subsitite(ans.split(), pat_to_dict(got_patterns)))
    except UnboundLocalError:
        return print('no matching rule')

get_response('My mother told me something',defined_patterns)

'How do you think about your mother ?'

## 复杂问题匹配

In [7]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

from collections import defaultdict

fail = [True, None]

def pat_match_with_seg(pattern, saying):
    
    global match_level_start
    
    if not pattern or not saying: return []
    
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])[0:-1] + [match_level_start]
    elif is_pattern_segment(pat):
        match, index, match_level = segment_match(pattern, saying)
        match_level_start *= match_level
#        print(match_level_start)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])[0:-1] + [match_level_start]
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])[0:-1] + [match_level_start]
    else:
        return fail

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying) ,1   
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i,1
    
    return (seg_pat, saying), len(saying),0

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

### 中文匹配

In [10]:
import jieba

def cut(string):return list(jieba.cut(string))

defined_patterns_chinese = {
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '可这话是?*x说的':['?x说的就一定是真的吗，你再重复一边就代表你也有责任吧',
                 '我从未见过?x这样的厚颜无耻之徒',
                 '?x你算计我!!',
                 '?x接我律师函警告']           
}

def string2rule(rule):
    string = []
    rule = cut(rule)
    for i,word in enumerate(rule):
        if i == 0:
            string.append(word)
        else:
            if '\u4e00' <= word <= '\u9fff' and (string[-1] > '\u9fff' or string[-1] < '\u4e00'):
                string.append(' ')
                string.append(word)
            elif '\u4e00' <= string[-1] <= '\u9fff' and ('\u4e00' <= word <= '\u9fff'):
                string.append(' ')
                string.append(word)
            elif (word > '\u9fff' or word < '\u4e00') and '\u4e00' <= string[-1] <= '\u9fff':
                string.append(' ')
                string.append(word)
            else:string.append(word)
    rule = ''.join(string)
    return rule
                
def get_response_chinese(saying_chinese,defined_patterns_chinese):
    saying_jieba = ' '.join(cut(saying_chinese))
#    print(saying)
    for rule in defined_patterns_chinese.keys():
        global match_level_start 
        match_level_start = 1
        rule_jieba = string2rule(rule)
        if pat_match_with_seg(rule_jieba.split(),saying_jieba.split()):
#            print(rule)
            ans = choice(defined_patterns_chinese[rule])
            ans_jieba = ''.join(string2rule(ans))
            got_patterns = pat_match_with_seg(rule_jieba.split(),saying_jieba.split())
#            print(got_patterns[-1])
            if got_patterns[-1] == 1:
                print(saying_chinese)
                break
    try:
        return ' '.join(subsitite(ans_jieba.split(), pat_to_dict(got_patterns[0:-1])))
    except UnboundLocalError:
        return print('no matching rule')

### 中文匹配结果

In [11]:
get_response_chinese('可这话是彭定康说的',defined_patterns_chinese).replace(' ','')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ric\AppData\Local\Temp\jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built succesfully.


可这话是彭定康说的


'彭定康说的就一定是真的吗，你再重复一边就代表你也有责任吧'